In [ ]:
%load_ext autoreload
%autoreload 2

# PriorVAE: testing MMD (and in general different loss functions)

For a 1D GP

In [ ]:
from jax import random
import jax.numpy as jnp

import time
import dill
from flax import serialization

# Numpyro
import numpyro
from numpyro.infer import MCMC, NUTS, init_to_median, Predictive


In [ ]:
numpyro.set_host_device_count(3)

In [ ]:
from reusable.kernels import esq_kernel, rbf_kernel

args = {
    # GP prior configuration
    "n": 100,
    "gp_kernel": esq_kernel,
    "rng_key": random.PRNGKey(2),
}
args.update({ # so we can use the definition of n to define x
    
    "x": jnp.arange(0, 1, 1/args["n"]),

    # VAE configuration
    "hidden_dim1": 35,
    "hidden_dim2": 32,
    "latent_dim": 30,
    "vae_var": 0.1,

    "mmd_kernel": lambda x,z: rbf_kernel(x,z, 4.0),

    # learning
    "num_epochs": 200,
    "learning_rate": 1.0e-4,
    "batch_size": 200,
    "train_num_batches": 500,
    "test_num_batches": 1,

    # MCMC parameters
    "num_warmup": 1000,
    "num_samples": 1000,
    "thinning": 1,
    "num_chains": 3,

    "pretrained_vae": False


})

rng_key, _ = random.split(random.PRNGKey(4))

## SVI to learn VAE parameters

In [ ]:
if not args["pretrained_vae"]:
    from reusable.gp import OneDGP
    from reusable.train_nn import gen_gp_batches

    
    rng_key, rng_key_train, rng_key_test = random.split(rng_key, 3)

    train_draws = gen_gp_batches(args["x"], OneDGP, args["gp_kernel"], args["train_num_batches"], args["batch_size"], rng_key_train)
    test_draws = gen_gp_batches(args["x"], OneDGP, args["gp_kernel"], 1, args["test_num_batches"]* args["batch_size"], rng_key_test)



In [ ]:
from reusable.vae import VAE
from reusable.train_nn import SimpleTrainState
import optax

rng_key, rng_key_init, rng_key_train = random.split(rng_key, 3)

module = VAE(
    hidden_dim1=args["hidden_dim1"],
    hidden_dim2=args["hidden_dim2"],
    latent_dim=args["latent_dim"],
    out_dim=args["n"],
    conditional=False,
)
params = module.init(rng_key, jnp.ones((args["n"],)))["params"]  # initialize parameters by passing a template image
tx = optax.adam(args["learning_rate"])
state = SimpleTrainState.create(apply_fn=module.apply, params=params, tx=tx, key=rng_key_init)


In [ ]:
import optax
import jax

from reusable.vae import vae_sample
from flax.core.frozen_dict import freeze
from functools import partial

from reusable.mmd import mmd

@jax.jit
def RCL(y, reconstructed_y, mean, log_sd):
    """reconstruction loss, averaged over the datapoints (not summed)"""
    return  jnp.sum(optax.l2_loss(reconstructed_y, y)) # 1/y.shape[0] *

@jax.jit
def KLD(y, reconstructed_y, mean, log_sd):
    """KL divergence between the distribution N(mean, log_sd) and a standard normal.
    e.g. see https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Multivariate_normal_distributions"""
    return -0.5 * jnp.sum(1 + log_sd - jnp.power(mean, 2) - jnp.exp(log_sd))

@jax.jit
def MMD(y, reconstructed_y, mean, log_sd):
    return mmd(y, reconstructed_y, args["mmd_kernel"])

@jax.jit
def rcl_kld(*args):
    return RCL(*args) + KLD(*args)

@jax.jit
def rcl_kld_50mmd(*args):
    return RCL(*args) + KLD(*args) + 50* MMD(*args)

@jax.jit
def kld_50mmd(*args):
    return 50* MMD(*args) + KLD(*args)


def compute_epoch_metrics(final_state: SimpleTrainState, test_samples, train_samples, train_output, test_output):

    current_metric_key = jax.random.fold_in(key=final_state.key, data=2 * final_state.step + 1)

    vae_draws = Predictive(vae_sample, num_samples=args["batch_size"])(
        current_metric_key,
        hidden_dim1=args["hidden_dim1"],
        hidden_dim2=args["hidden_dim2"],
        latent_dim=args["latent_dim"],
        out_dim=args["n"],
        decoder_params=freeze({"params": final_state.params["VAE_Decoder_0"]}),
    )["f"]

    metrics = {
        "train_mmd": mmd(vae_draws, train_samples[-1], args["mmd_kernel"]),
        "test_mmd": mmd(vae_draws, test_samples[-1], args["mmd_kernel"]),
        "train_kld": KLD(*train_output),
        "test_kld": KLD(*test_output),
        "train_rcl": RCL(*train_output),
        "test_rcl": RCL(*test_output)
    }

    return metrics


In [11]:
from reusable.train_nn import run_training
import matplotlib.pyplot as plt
from plotting.plots import plot_training

from reusable.vae import vae_sample
from plotting.plots import compare_draws
import jax.random as random
from numpyro.infer import Predictive
from reusable.gp import OneDGP
from reusable.util import decoder_filename, get_savepath



_, rng_key_predict = random.split(random.PRNGKey(2))

plot_gp_predictive = Predictive(OneDGP, num_samples=1000)
gp_draws = plot_gp_predictive(rng_key_predict, x=args["x"], gp_kernel = args["gp_kernel"], jitter=1e-5)['y']


saved_states  = {}
if not args["pretrained_vae"]:
    loss_fns = [rcl_kld_50mmd, kld_50mmd]
    for loss_fn in loss_fns:
        print( loss_fn.__name__)
        final_state, metrics_history = run_training(
            loss_fn, compute_epoch_metrics, args["num_epochs"], train_draws, test_draws, state
        )
        saved_states[loss_fn.__name__] = final_state
        fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(24, 4))
        for i,t in enumerate(["loss", "mmd", "kld", "rcl"]):

            plot_training(
                jnp.array(metrics_history["test_"+t]).flatten(),
                jnp.array(metrics_history["train_"+t]).flatten(),
                f"Test/train {t} for " + loss_fn.__name__,
                t,
                axs[i],
            )
        plt.show()
        print(metrics_history)
        plot_vae_predictive = Predictive(vae_sample, num_samples=1000)
        vae_draws = plot_vae_predictive(
            rng_key_predict,
            hidden_dim1=args["hidden_dim1"],
            hidden_dim2=args["hidden_dim2"],
            latent_dim=args["latent_dim"],
            out_dim=args["n"],
            decoder_params=freeze({"params": final_state.params["VAE_Decoder_0"]}),
        )["f"]
            


        compare_draws(args["x"], gp_draws, vae_draws, "GP priors we want to encode", "Priors learnt by VAE w/ loss" + loss_fn.__name__, '$y=f_{GP}(x)$', '$y=f_{VAE}(x)$', save_path="gen_plots/01_prior_comp.png")
        plt.show()

        file_path = f'{get_savepath()}/{decoder_filename("04", args, suffix=loss_fn.__name__)}'

        if not args["pretrained_vae"]:
            decoder_params = state.params["VAE_Decoder_0"]

            decoder_params = freeze({"params": decoder_params})
            args["decoder_params"] = decoder_params
            with open(file_path, 'wb') as file:
                file.write(serialization.to_bytes(decoder_params))

        file_path = f'{get_savepath()}/{decoder_filename("04", args, suffix=loss_fn.__name__+"_metrics_hist")}'

        with open(file_path, 'wb') as file:
            file.write(dill.dump(metrics_history))



Note 
that the jax fori_loop MMD definition is the same as the original one that generates the kernel matrices


(Run the following cell to check - takes lots of time and memory!)

In [ ]:
# from reusable.mmd import mmd, orig_mmd
#print(MMD(train_draws[1], test_draws[0], 0,0))

# print(orig_mmd(train_draws[1], test_draws[0], args["mmd_kernel"]))